In [ ]:
from datasets import load_dataset
import gc
from peft import LoraConfig, TaskType
import torch
from tqdm.auto import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer
from trl import SFTConfig, SFTTrainer
import trl

In [36]:
dataset = load_dataset("json", data_files={"train": "./train.json", "test": "./test.json"})


train_dataset = dataset["train"]
test_dataset = dataset["test"]

print(train_dataset)
print(train_dataset[0])
print(test_dataset)
print(test_dataset[0])

Dataset({
    features: ['completion', 'prompt'],
    num_rows: 51
})
{'completion': [{'content': 'Odpowiadaj krótko i konwersacyjnie :)', 'role': 'system'}, {'content': 'Jak mogę pomóc?', 'role': 'user'}, {'content': 'Jestem tutaj, aby pomóc. Jeśli masz jakieś pytania lub potrzebujesz wsparcia, śmiało pytaj!', 'role': 'assistant'}], 'prompt': [{'content': 'Odpowiadaj krótko i konwersacyjnie :)', 'role': 'system'}, {'content': 'Jak mogę pomóc?', 'role': 'user'}]}
Dataset({
    features: ['completion', 'prompt'],
    num_rows: 13
})
{'completion': [{'content': 'Odpowiadaj krótko i konwersacyjnie :)', 'role': 'system'}, {'content': 'Jak działa sieć neuronowa?', 'role': 'user'}, {'content': 'Sieć neuronowa to model matematyczny inspirowany strukturą i funkcjami biologicznych neuronów.', 'role': 'assistant'}], 'prompt': [{'content': 'Odpowiadaj krótko i konwersacyjnie :)', 'role': 'system'}, {'content': 'Jak działa sieć neuronowa?', 'role': 'user'}]}


In [ ]:
torch.cuda.empty_cache()
gc.collect()

tokenizer = AutoTokenizer.from_pretrained("sdadas/polish-gpt2-small")
bielik_tokenizer = AutoTokenizer.from_pretrained("speakleash/Bielik-1.5B-v3.0-Instruct")
model = AutoModelForCausalLM.from_pretrained("sdadas/polish-gpt2-small")

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["c_attn", "c_fc", "c_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM 
)

trl.clone_chat_template(model, tokenizer, source_tokenizer_path="speakleash/Bielik-1.5B-v3.0-Instruct")

training_args = SFTConfig(
    output_dir="student_model",
    max_length=512,
    assistant_only_loss=True,
    completion_only_loss=True,
    per_device_train_batch_size=8, 
    gradient_accumulation_steps=1,
    remove_unused_columns=False,
    learning_rate=1e-5,
    eval_strategy="steps",
    num_train_epochs=5

)

trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    processing_class=tokenizer,
    peft_config=lora_config
)

trainer.train()

/home/jakub/Coding/30 - Personal projects/Self-reflect MiniGPT/.venv/lib/python3.13/site-packages/peft/tuners/lora/layer.py:1803: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss,Validation Loss
10,7.089400,6.314740
20,6.844700,6.020119
30,6.609700,5.909581


/home/jakub/Coding/30 - Personal projects/Self-reflect MiniGPT/.venv/lib/python3.13/site-packages/peft/utils/save_and_load.py:252: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


TrainOutput(global_step=35, training_loss=6.809966278076172, metrics={'train_runtime': 6.2618, 'train_samples_per_second': 40.723, 'train_steps_per_second': 5.589, 'total_flos': 9926068027392.0, 'train_loss': 6.809966278076172})

In [ ]:
print(test_dataset[0])
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
out = tokenizer.apply_chat_template(test_dataset[0]["prompt"], add_generation_prompt=True, return_tensors="pt").to(DEVICE)
print(out)
tokenizer.decode(model.generate(out)[0], pad_token_id=tokenizer.eos_token_id)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:51201 for open-end generation.


{'completion': [{'content': 'Odpowiadaj krótko i konwersacyjnie :)', 'role': 'system'}, {'content': 'Jak działa sieć neuronowa?', 'role': 'user'}, {'content': 'Sieć neuronowa to model matematyczny inspirowany strukturą i funkcjami biologicznych neuronów.', 'role': 'assistant'}], 'prompt': [{'content': 'Odpowiadaj krótko i konwersacyjnie :)', 'role': 'system'}, {'content': 'Jak działa sieć neuronowa?', 'role': 'user'}]}
tensor([[    0, 51200, 22283,   203, 16503,    78,  5879,   292, 36004,  6175,
          7908, 51201,   203, 51200,    89,  1446,   203,   880,   878, 11770,
         10938,  3385,    35, 51201,   203, 51200,   400,   393,  1111,    88,
           203]], device='cuda:0')


'<s><|im_start|>system\nOdpowiadaj krótko i konwersacyjnie :)<|im_end|>\n<|im_start|>user\nJak działa sieć neuronowa?<|im_end|>\n<|im_start|>assistant\n.</s>W tym roku w Polsce jest ponad 1,5 mln osób, które nie ukończyły 18'

: 